In [1]:
import fluidsf
import h5py
import numpy as np

In [2]:
%%time
# pathname = '../simulations/output/copied_data/2D_Equilibrated/'
# filename = 'Anisotropic2D_n_2048_drag_0.04_order_-2_visc_1.0e-21_order_8_kf_100.0_F_1.0e-5_betay_10.0_betax_0.0_3000.mat'
pathname = '../simulations/output/copied_data/SurfaceQG_n_512_visc_1.0e-17_order_8_hypovisc_0.005_order_-2_kf_7.0_F_1.0e-5_endtime_30000.0_beta_0.5/'
filename = 'SurfaceQG_n_512_visc_1.0e-17_order_8_hypovisc_0.005_order_-2_kf_7.0_F_1.0e-5_endtime_30000.0_beta_0.5_30040.mat'

f = h5py.File(pathname + filename, "r")

x = np.linspace(-f['Lx'][0]/2, f['Lx'][0]/2, f['nx'][0])
y = np.linspace(-f['Ly'][0]/2, f['Ly'][0]/2, f['ny'][0])

CPU times: user 1.76 ms, sys: 1.2 ms, total: 2.96 ms
Wall time: 2.15 ms


In [3]:
%%time

sfs = fluidsf.generate_structure_functions_2d(f['u'], f['v'], x, y, 
                                              sf_type=["ASF_V"])

CPU times: user 5.39 s, sys: 2.58 s, total: 7.97 s
Wall time: 7.97 s


In [4]:
%%time

sfs = fluidsf.generate_structure_functions_2d(f['u'], f['v'], x, y, 
                                              scalar=f['b'], 
                                              sf_type=["ASF_V","ASF_S","LLL","TTT","LTT","LSS"])

CPU times: user 22.4 s, sys: 9.08 s, total: 31.5 s
Wall time: 31.5 s


In [ ]:
%%time

sf_maps = fluidsf.generate_sf_maps_2d(np.array(f['u']), np.array(f['v']), x, y, 
                                              scalar=np.array(f['b']), 
                                              sf_type=["ASF_V","ASF_S","LLL","TTT","LTT","LSS"])

/Users/brodiepearson/opt/miniconda3/lib/python3.8/site-packages/fluidsf/calculate_sf_maps_2d.py:115: RuntimeWarning: invalid value encountered in scalar divide
  cosine_angle = x_separation / np.sqrt(x_separation**2 + y_separation**2)
/Users/brodiepearson/opt/miniconda3/lib/python3.8/site-packages/fluidsf/calculate_sf_maps_2d.py:116: RuntimeWarning: invalid value encountered in scalar divide
  sine_angle = y_separation / np.sqrt(x_separation**2 + y_separation**2)
/Users/brodiepearson/opt/miniconda3/lib/python3.8/site-packages/fluidsf/calculate_sf_maps_2d.py:119: RuntimeWarning: Mean of empty slice
  SF_dict["SF_LL_xy"] = np.nanmean(
/Users/brodiepearson/opt/miniconda3/lib/python3.8/site-packages/fluidsf/calculate_sf_maps_2d.py:127: RuntimeWarning: Mean of empty slice
  SF_dict["SF_TT_xy"] = np.nanmean(
/Users/brodiepearson/opt/miniconda3/lib/python3.8/site-packages/fluidsf/calculate_sf_maps_2d.py:138: RuntimeWarning: Mean of empty slice
  SF_dict["SF_LLL_xy"] = np.nanmean(
/Users/brodi

In [ ]:
np.array(f['u'])

In [ ]:
import pickle

with open('2D_sf_maps.pkl', 'wb') as fp:
    pickle.dump(sfs, fp)
    
# Read dictionary pkl file
with open('2D_sf_maps.pkl', 'rb') as fp:
    sfs_new = pickle.load(fp)
    
sfs, sfs_new

In [ ]:
import matplotlib.pyplot as plt

plt.semilogx(sfs['x-diffs'], sfs['SF_advection_velocity_x'])
plt.semilogx(sfs['y-diffs'], sfs['SF_advection_velocity_y'])

In [ ]:
sfs['x-diffs'].max()

In [ ]:
pathname = './processed_data/'

filename = 'Structure_Functions_2D_StrongBeta_Zonal.mat'
sfs_matlab_zonal = h5py.File(pathname + filename, "r")

filename = 'Structure_Functions_2D_StrongBeta_Meridional.mat'
sfs_matlab_meridional = h5py.File(pathname + filename, "r")

separation_distances = np.transpose(np.array(sfs_matlab_zonal['R']))

Advective_SF_Zonal_snapshots = sfs_matlab_zonal['uuadv_snapshots'][:,:]+ sfs_matlab_zonal['vvadv_snapshots'][:,:]
Advective_SF_Meridional_snapshots = sfs_matlab_meridional['uuadv_snapshots'][:,:]+ sfs_matlab_meridional['vvadv_snapshots'][:,:]

In [ ]:
import matplotlib.pyplot as plt

time_index = -15 # Why 15th time snapshot? No idea! Probably something funky with Matlab's datenum time conversions

plt.semilogx(sfs['x-diffs'], sfs['SF_advection_velocity_x'], 'k')
plt.semilogx(sfs['y-diffs'], sfs['SF_advection_velocity_y'], 'r')
plt.semilogx(separation_distances, Advective_SF_Zonal_snapshots[time_index,:], 'k--')
plt.semilogx(separation_distances, Advective_SF_Meridional_snapshots[time_index,:], 'r--')


plt.semilogx(separation_distances, np.transpose(sfs_matlab_zonal['uuadv'][:]+ sfs_matlab_zonal['vvadv'][:]), 'k:')
plt.semilogx(separation_distances, np.transpose(sfs_matlab_meridional['uuadv'][:]+ sfs_matlab_meridional['vvadv'][:]), 'r:')

In [ ]:

plt.plot(sfs['x-diffs'], sfs['SF_advection_velocity_x'], 'k')
plt.plot(sfs['y-diffs'], sfs['SF_advection_velocity_y'], 'r')
plt.semilogx(separation_distances, Advective_SF_Zonal_snapshots[time_index,:], 'k--')
plt.semilogx(separation_distances, Advective_SF_Meridional_snapshots[time_index,:], 'r--')

plt.plot(separation_distances, np.transpose(sfs_matlab_zonal['uuadv'][:]+ sfs_matlab_zonal['vvadv'][:]), 'k:')
plt.plot(separation_distances, np.transpose(sfs_matlab_meridional['uuadv'][:]+ sfs_matlab_meridional['vvadv'][:]), 'r:')

In [ ]:

plt.semilogx(separation_distances, np.transpose(sfs_matlab_zonal['uuadv'][:]+ sfs_matlab_zonal['vvadv'][:]), 'k:')
plt.semilogx(separation_distances, np.transpose(sfs_matlab_meridional['uuadv'][:]+ sfs_matlab_meridional['vvadv'][:]), 'r:')